In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from scipy.stats import pearsonr
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import calendar
from matplotlib.ticker import FuncFormatter
import datetime
import joblib
import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
others = pd.read_csv('almeda_data.csv')

other = others
others.head()

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,...,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed,Global Horizontal UV Irradiance (280-400nm),Global Horizontal UV Irradiance (295-385nm)
0,2021,1,1,0,30,6.7,0,0,0,0,...,0,98.33,164.94,0.16,1026,1.5,318,1.2,0.0000,0.0000
1,2021,1,1,1,30,7.0,0,0,0,0,...,0,95.58,157.77,0.16,1026,1.5,309,1.1,0.0000,0.0000
2,2021,1,1,10,30,14.4,70,831,422,0,...,422,52.85,64.97,0.16,1027,1.3,197,1.2,22.5139,17.5246
3,2021,1,1,11,30,16.4,76,863,492,3,...,395,38.49,61.20,0.16,1027,1.3,242,1.1,21.6196,16.8738
4,2021,1,1,12,30,17.0,80,854,500,7,...,381,35.19,60.55,0.16,1027,1.4,265,1.1,20.9364,16.3469


In [3]:
column_data_types = others.dtypes

# Print the data types
print("Data types of each column in the in predictors DataFrame:")
print(column_data_types)

Data types of each column in the in predictors DataFrame:
Year                                             int64
Month                                            int64
Day                                              int64
Hour                                             int64
Minute                                           int64
Temperature                                    float64
Clearsky DHI                                     int64
Clearsky DNI                                     int64
Clearsky GHI                                     int64
Cloud Type                                       int64
Dew Point                                      float64
DHI                                              int64
DNI                                              int64
Fill Flag                                        int64
GHI                                              int64
Relative Humidity                              float64
Solar Zenith Angle                             float64
Surface

In [4]:
energy_data = pd.read_excel('Almeda Electrical Info 1.xlsx')
energy_data.head()

,end of billing period,Electrical,On-Peak Usage (kWh),Part-Peak Usage (kWh),Off-Peak Usage (kWh),Total Usage (kWh),Charges (dollars),Billing Demand (kW),Created Demand (kW),On-Peak Demand (kW),Part-Peak Demand (kW),Off-Peak Demand (kW)
0,2023-06-01,87810.22,NaN,NaN,NaN,393447.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-05-01,82891.88,NaN,NaN,NaN,428607.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-04-01,68746.43,NaN,NaN,NaN,362335.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-03-01,68659.27,NaN,NaN,NaN,352314.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-01,73939.40,NaN,NaN,NaN,372816.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
column_data_types = energy_data.dtypes

# Print the data types
print("Data types of each column in the in Energy Consumption DataFrame:")
print(column_data_types)

Data types of each column in the in Energy Consumption DataFrame:
end of billing period    datetime64[ns]
Electrical                      float64
On-Peak Usage (kWh)             float64
Part-Peak Usage (kWh)           float64
Off-Peak Usage (kWh)            float64
Total Usage (kWh)               float64
Charges (dollars)               float64
Billing Demand (kW)             float64
Created Demand (kW)             float64
On-Peak Demand (kW)             float64
Part-Peak Demand (kW)           float64
Off-Peak Demand (kW)            float64
dtype: object


In [6]:
flow_data = pd.read_excel('Almeda Flows.xlsx')
flow_data.head()

,Date,BPW Flow Average (MGD)
0,2013-06-22,9.81
1,2013-06-23,9.81
2,2013-06-24,9.82
3,2013-06-25,8.81
4,2013-06-26,8.64


In [7]:
rows, columns = flow_data.shape
print(f"flow_data has {rows} rows and {columns} columns.")

flow_data has 3650 rows and 2 columns.


In [8]:
column_data_types = flow_data.dtypes

# Print the data types
print("Data types of each column in the in Water Flow DataFrame:")
print(column_data_types)

Data types of each column in the in Water Flow DataFrame:
Date                      datetime64[ns]
BPW Flow Average (MGD)           float64
dtype: object


In [9]:
# Convert Month, Day, and Year columns to a datetime format
others['Date'] = pd.to_datetime(others['Year'].astype(str) + '-' + others['Month'].astype(str) + '-' + others['Day'].astype(str), errors='coerce')

other['Date'] = pd.to_datetime(other['Year'].astype(str) + '-' + other['Month'].astype(str) + '-' + other['Day'].astype(str), errors='coerce')



In [10]:
others = pd.get_dummies(others, columns=['Month'], drop_first=True)

# Convert 'Day of the Month' to integer if it's an object
if others['Day'].dtype == 'object':
    others['Day'] = others['Day'].astype(int)


In [11]:
column_data_types = others.dtypes

# Print the data types
print("Data types of each column in the in predictors DataFrame:")
print(column_data_types)

Data types of each column in the in predictors DataFrame:
Year                                                    int64
Day                                                     int64
Hour                                                    int64
Minute                                                  int64
Temperature                                           float64
Clearsky DHI                                            int64
Clearsky DNI                                            int64
Clearsky GHI                                            int64
Cloud Type                                              int64
Dew Point                                             float64
DHI                                                     int64
DNI                                                     int64
Fill Flag                                               int64
GHI                                                     int64
Relative Humidity                                     float64
Solar Zenith

In [12]:
# Check for duplicate dates in flow_data and others
flow_duplicates = flow_data[flow_data['Date'].duplicated()]
others_duplicates = others[others['Date'].duplicated()]

print(f"Number of duplicate dates in flow_data: {len(flow_duplicates)}")
print(f"Number of duplicate dates in others: {len(others_duplicates)}")

# Drop duplicate dates in both dataframes (keep the first occurrence)
flow_data = flow_data.drop_duplicates(subset='Date', keep='first')
others = others.drop_duplicates(subset='Date', keep='first')

# Merge the dataframes on the 'Date' column again
flow_data = pd.merge(flow_data, others, on='Date', how='inner')

print(f"Number of rows in the merged dataframe: {len(flow_data)}")


Number of duplicate dates in flow_data: 0
Number of duplicate dates in others: 100740
Number of rows in the merged dataframe: 3111


In [13]:
rows, columns = flow_data.shape
print(f"flow_data has {rows} rows and {columns} columns.")

flow_data has 3111 rows and 36 columns.


In [14]:
# Display the first few rows of the modified dataframe
print(flow_data.head())


        Date  BPW Flow Average (MGD)  Year  Day  Hour  Minute  Temperature  \
0 2013-06-22                    9.81  2013   22     0      30         12.5   
1 2013-06-23                    9.81  2013   23     0      30         15.1   
2 2013-06-24                    9.82  2013   24     0      30         16.3   
3 2013-06-25                    8.81  2013   25     0      30         16.4   
4 2013-06-26                    8.64  2013   26     0      30         17.7   

   Clearsky DHI  Clearsky DNI  Clearsky GHI  ...  Month_3  Month_4  Month_5  \
0             0             0             0  ...        0        0        0   
1             0             0             0  ...        0        0        0   
2             0             0             0  ...        0        0        0   
3             0             0             0  ...        0        0        0   
4             0             0             0  ...        0        0        0   

   Month_6  Month_7  Month_8  Month_9  Month_10  Month_1

In [15]:
# Check for missing values in each column of flow_data
missing_values = flow_data.isna().sum()
print("Missing values in each column before removal:")
print(missing_values)

Missing values in each column before removal:
Date                                           0
BPW Flow Average (MGD)                         0
Year                                           0
Day                                            0
Hour                                           0
Minute                                         0
Temperature                                    0
Clearsky DHI                                   0
Clearsky DNI                                   0
Clearsky GHI                                   0
Cloud Type                                     0
Dew Point                                      0
DHI                                            0
DNI                                            0
Fill Flag                                      0
GHI                                            0
Relative Humidity                              0
Solar Zenith Angle                             0
Surface Albedo                                 0
Pressure               

In [16]:
column_data_types = flow_data.dtypes

# Print the data types
print("Data types of each column in the in Water Flow DataFrame:")
print(column_data_types)

Data types of each column in the in Water Flow DataFrame:
Date                                           datetime64[ns]
BPW Flow Average (MGD)                                float64
Year                                                    int64
Day                                                     int64
Hour                                                    int64
Minute                                                  int64
Temperature                                           float64
Clearsky DHI                                            int64
Clearsky DNI                                            int64
Clearsky GHI                                            int64
Cloud Type                                              int64
Dew Point                                             float64
DHI                                                     int64
DNI                                                     int64
Fill Flag                                               int64
GHI         

In [17]:
# Extract 'Total Usage (kWh)' and 'end of billing period' columns
energy_data = energy_data[['Total Usage (kWh)', 'end of billing period']]

# Rename 'end of billing period' to 'Date'
energy_data = energy_data.rename(columns={'end of billing period': 'Date'})

# Display the first few rows of the modified DataFrame
print(energy_data.head())


   Total Usage (kWh)       Date
0           393447.0 2023-06-01
1           428607.0 2023-05-01
2           362335.0 2023-04-01
3           352314.0 2023-03-01
4           372816.0 2023-02-01


In [18]:
# Merge the dataframes on the 'Date' column
energy_data = pd.merge(energy_data, other, on='Date', how='inner')

# Display the first few rows of the merged dataframe
print(energy_data.head())

   Total Usage (kWh)       Date  Year  Month  Day  Hour  Minute  Temperature  \
0            51235.0 2021-12-01  2021     12    1     0      30         11.0   
1            51235.0 2021-12-01  2021     12    1     1      30         10.6   
2            51235.0 2021-12-01  2021     12    1    10      30         20.3   
3            51235.0 2021-12-01  2021     12    1    11      30         22.7   
4            51235.0 2021-12-01  2021     12    1    12      30         24.2   

   Clearsky DHI  Clearsky DNI  ...  GHI  Relative Humidity  \
0             0             0  ...    0              97.70   
1             0             0  ...    0             100.00   
2            71           863  ...  465              60.48   
3            74           890  ...  522              51.60   
4            73           892  ...  520              47.36   

   Solar Zenith Angle  Surface Albedo  Pressure  Precipitable Water  \
0              162.81            0.15      1026                 1.4   
1   

In [19]:
# Display the first few rows of the modified dataframe
print(energy_data.head())


   Total Usage (kWh)       Date  Year  Month  Day  Hour  Minute  Temperature  \
0            51235.0 2021-12-01  2021     12    1     0      30         11.0   
1            51235.0 2021-12-01  2021     12    1     1      30         10.6   
2            51235.0 2021-12-01  2021     12    1    10      30         20.3   
3            51235.0 2021-12-01  2021     12    1    11      30         22.7   
4            51235.0 2021-12-01  2021     12    1    12      30         24.2   

   Clearsky DHI  Clearsky DNI  ...  GHI  Relative Humidity  \
0             0             0  ...    0              97.70   
1             0             0  ...    0             100.00   
2            71           863  ...  465              60.48   
3            74           890  ...  522              51.60   
4            73           892  ...  520              47.36   

   Solar Zenith Angle  Surface Albedo  Pressure  Precipitable Water  \
0              162.81            0.15      1026                 1.4   
1   

In [20]:
# Check for missing values in each column of flow_data
missing_values = energy_data.isna().sum()
print("Missing values in each column before removal:")
print(missing_values)

# Check if there are any missing values in the DataFrame
if missing_values.any():
    # Drop rows with missing values
    energy_data.dropna(inplace=True)
    print("Rows with missing values have been removed.")

# Verify that there are no more missing values
missing_values_after = energy_data.isna().sum()
print("Missing values in each column after removal:")
print(missing_values_after)


Missing values in each column before removal:
Total Usage (kWh)                              336
Date                                             0
Year                                             0
Month                                            0
Day                                              0
Hour                                             0
Minute                                           0
Temperature                                      0
Clearsky DHI                                     0
Clearsky DNI                                     0
Clearsky GHI                                     0
Cloud Type                                       0
Dew Point                                        0
DHI                                              0
DNI                                              0
Fill Flag                                        0
GHI                                              0
Relative Humidity                                0
Solar Zenith Angle                  

In [21]:
# Number of lags to create
n_lags = 3

# Create lagged features for "Water Flows"
for i in range(1, n_lags + 1):
    col_name = f'Water_Flows_Lag_{i}'
    flow_data[col_name] = flow_data['BPW Flow Average (MGD)'].shift(i)

# Remove the rows with NaN values generated by shifting
flow_data.dropna(inplace=True)

# Show the first few rows to confirm
print(flow_data.head())

        Date  BPW Flow Average (MGD)  Year  Day  Hour  Minute  Temperature  \
3 2013-06-25                    8.81  2013   25     0      30         16.4   
4 2013-06-26                    8.64  2013   26     0      30         17.7   
5 2013-06-27                   10.21  2013   27     0      30         16.7   
6 2013-06-28                   10.70  2013   28     0      30         18.1   
7 2013-06-29                   10.71  2013   29     0      30         19.2   

   Clearsky DHI  Clearsky DNI  Clearsky GHI  ...  Month_6  Month_7  Month_8  \
3             0             0             0  ...        1        0        0   
4             0             0             0  ...        1        0        0   
5             0             0             0  ...        1        0        0   
6             0             0             0  ...        1        0        0   
7             0             0             0  ...        1        0        0   

   Month_9  Month_10  Month_11  Month_12  Water_Flows_La

In [22]:
# Number of lags to create
n_lags = 3

# Create lagged features for "Water Flows"
for i in range(1, n_lags + 1):
    col_name = f'Energy_Lag_{i}'
    energy_data[col_name] = energy_data['Total Usage (kWh)'].shift(i)

# Remove the rows with NaN values generated by shifting
energy_data.dropna(inplace=True)

# Show the first few rows to confirm
print(energy_data.head())

   Total Usage (kWh)       Date  Year  Month  Day  Hour  Minute  Temperature  \
3            51235.0 2021-12-01  2021     12    1    11      30         22.7   
4            51235.0 2021-12-01  2021     12    1    12      30         24.2   
5            51235.0 2021-12-01  2021     12    1    13      30         24.9   
6            51235.0 2021-12-01  2021     12    1    14      30         24.7   
7            51235.0 2021-12-01  2021     12    1    15      30         24.4   

   Clearsky DHI  Clearsky DNI  ...  Surface Albedo  Pressure  \
3            74           890  ...            0.15      1026   
4            73           892  ...            0.15      1025   
5            68           867  ...            0.15      1025   
6            59           801  ...            0.15      1024   
7            46           655  ...            0.15      1024   

   Precipitable Water  Wind Direction  Wind Speed  \
3                 1.1              93         1.8   
4                 1.1       

In [23]:
# Split sizes for Treated Water
train_size = int(len(flow_data) * 0.5)  # 50% for training

# Train-Test Split
train, test = flow_data[:train_size], flow_data[train_size:]

# Prepare data for machine learning models for actual values

# Training data
X_train_actual = train.drop(['Date', 'BPW Flow Average (MGD)'], axis=1)
y_train_treated_actual = train['BPW Flow Average (MGD)']

# Test data
X_test_actual = test.drop(['Date', 'BPW Flow Average (MGD)'], axis=1)
y_test_treated_actual = test['BPW Flow Average (MGD)']

print("Training set size:", len(X_train_actual))
print("Test set size:", len(X_test_actual))

Training set size: 1554
Test set size: 1554


In [24]:
# Split sizes for Energy Consumption
train_size_energy = int(len(energy_data) * 0.5)  # 50% for training

# Train-Test Split
train_energy, test_energy = energy_data[:train_size_energy], energy_data[train_size_energy:]

# Prepare data for machine learning models for actual values

# Training data
X_train_actual_energy = train_energy.drop(['Date', 'Total Usage (kWh)'], axis=1)
y_train_actual_energy = train_energy['Total Usage (kWh)']

# Test data
X_test_actual_energy = test_energy.drop(['Date', 'Total Usage (kWh)'], axis=1)
y_test_actual_energy = test_energy['Total Usage (kWh)']

print("Training set size:", len(X_train_actual_energy))
print("Test set size:", len(X_test_actual_energy))


Training set size: 622
Test set size: 623


In [25]:
# Importing required libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt

In [26]:
# Machine Learning Model Training for Treated Water
rf_model_treated = RandomForestRegressor()
rf_model_treated.fit(X_train_actual, y_train_treated_actual)

gb_model_treated = GradientBoostingRegressor()
gb_model_treated.fit(X_train_actual, y_train_treated_actual)

# SARIMA Model Training for Treated Water
try:
    sarima_model_treated = SARIMAX(y_train_treated_actual, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    sarima_fit_treated = sarima_model_treated.fit(disp=False)
except Exception as e:
    print("Error fitting SARIMA model for Treated Water:", str(e))

C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [27]:
# Machine Learning Model Training for Energy Consumption
rf_model_energy = RandomForestRegressor()
rf_model_energy.fit(X_train_actual_energy, y_train_actual_energy)

gb_model_energy = GradientBoostingRegressor()
gb_model_energy.fit(X_train_actual_energy, y_train_actual_energy)


# SARIMA Model Training for Energy Consumption
try:
    sarima_model_energy = SARIMAX(y_train_actual_energy, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    sarima_fit_energy = sarima_model_energy.fit(disp=False)
except Exception as e:
    print("Error fitting SARIMA model for Energy Consumption:", str(e))

C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


In [28]:
# Predictions using Random Forest
rf_pred_flow = rf_model_treated.predict(X_test_actual)

# Predictions using Gradient Boosting
gb_pred_flow = gb_model_treated.predict(X_test_actual)

# Predictions using SARIMA
start_point_flow = len(y_train_treated_actual)
end_point_flow = start_point_flow + len(X_test_actual) - 1
sarima_pred_flow = sarima_fit_treated.predict(start=start_point_flow, end=end_point_flow)

# Ensemble Predictions for Treated Water
flow_outputs = [rf_pred_flow, gb_pred_flow, sarima_pred_flow]
ensemble_pred_flow = np.mean(flow_outputs, axis=0)

C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [29]:
# Predictions using Random Forest
rf_pred_energy = rf_model_energy.predict(X_test_actual_energy)

# Predictions using Gradient Boosting
gb_pred_energy = gb_model_energy.predict(X_test_actual_energy)

# Predictions using SARIMA Model
start_point_energy = len(y_train_actual_energy)
end_point_energy = start_point_energy + len(X_test_actual_energy) - 1
sarima_pred_energy = sarima_fit_energy.predict(start=start_point_energy, end=end_point_energy)

# Ensemble Predictions for Energy Consumption
energy_outputs = [rf_pred_energy, gb_pred_energy, sarima_pred_energy]
ensemble_pred_energy = np.mean(energy_outputs, axis=0)


C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [30]:
def error(y_true, y_pred, threshold=0.5):
    """
    Calculate the error rate based on a given threshold.
    The function returns the proportion of predictions that are off by more than the threshold.
    """
    return np.mean(np.abs(y_true - y_pred) > threshold)

In [31]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import numpy as np

def evaluate_predictions(y_true, y_pred, model_name):
    """
    Evaluate and print RMSE, MAE, and Error Rate for a given model's predictions.
    """
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    err_rate = error(y_true, y_pred)
    
    print(f"=== {model_name} ===")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"Error Rate: {err_rate:.4f}\n")
    return rmse, mae, err_rate

def main():
    # Evaluating Water Flow Predictions
    print("Evaluations for Water Flow:\n")
    evaluate_predictions(y_test_treated_actual, rf_pred_flow, "Random Forest")
    evaluate_predictions(y_test_treated_actual, gb_pred_flow, "Gradient Boosting")
    evaluate_predictions(y_test_treated_actual, sarima_pred_flow, "SARIMA")
    evaluate_predictions(y_test_treated_actual, ensemble_pred_flow, "Ensemble")
    
    # Evaluating Energy Consumption Predictions
    print("Evaluations for Energy Consumption:\n")
    evaluate_predictions(y_test_actual_energy, rf_pred_energy, "Random Forest")
    evaluate_predictions(y_test_actual_energy, gb_pred_energy, "Gradient Boosting")
    evaluate_predictions(y_test_actual_energy, sarima_pred_energy, "SARIMA")
    evaluate_predictions(y_test_actual_energy, ensemble_pred_energy, "Ensemble")

# Run the evaluations
main()

Evaluations for Water Flow:

=== Random Forest ===
RMSE: 1.4928
MAE: 0.8751
Error Rate: 0.4228

=== Gradient Boosting ===
RMSE: 1.7227
MAE: 1.1551
Error Rate: 0.6203

=== SARIMA ===
RMSE: 2.8473
MAE: 2.5797
Error Rate: 0.9345

=== Ensemble ===
RMSE: 1.8089
MAE: 1.4278
Error Rate: 0.8687

Evaluations for Energy Consumption:

=== Random Forest ===
RMSE: 160245.6473
MAE: 87844.1298
Error Rate: 1.0000

=== Gradient Boosting ===
RMSE: 168746.1546
MAE: 100129.3394
Error Rate: 1.0000

=== SARIMA ===
RMSE: 744929.6714
MAE: 696573.4417
Error Rate: 0.4438

=== Ensemble ===
RMSE: 276206.4898
MAE: 257052.1292
Error Rate: 1.0000

